In [ ]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(dplyr))

In [ ]:
obj.atac <- readRDS("../../DataIntegration/data/VisiumHeart/snATAC.annotated.Rds")
obj.rna <- readRDS("../../../snRNA/from_rico/integrated_snrnaseq/integrated_rnasamples_ann.rds")

In [ ]:
obj.atac

In [ ]:
obj.rna

In [ ]:
## update the same cell name
obj.rna$cell_type <- stringr::str_replace_all(obj.rna$cell_type,
                                              c("PC" = "Pericyte"))

In [ ]:
obj.rna$Sample <- obj.rna$orig.ident

In [ ]:
length(unique(obj.rna$Sample))
length(unique(obj.atac$Sample))

In [ ]:
table(obj.atac$cell_type)
table(obj.rna$cell_type)

In [ ]:
cols <- ArchR::paletteDiscrete(obj.rna$cell_type)

In [ ]:
p1 <- DimPlot(obj.rna, reduction = "umap_harmony", group.by = "cell_type",
             raster = TRUE, size = 0.1) +
    scale_color_manual(values = cols) +
    ggtitle("snRNA-seq")
    
p2 <- DimPlot(obj.atac, reduction = "umap_harmony_v2", group.by = "cell_type",
             size = 0.1, raster = TRUE) +
    scale_color_manual(values = cols) +
    ggtitle("snATAC-seq")

options(repr.plot.height = 5, repr.plot.width = 12)

p1 + p2

In [ ]:
## remove prolif. cells from snRNA
Idents(obj.rna) <- "cell_type"
obj.rna <- subset(obj.rna, idents = "prolif", invert = TRUE)
obj.rna

In [ ]:
# add meta data for snRNA-seq
df <- read.csv("../../../Annotations_figures/snrna_patient_anns_revisions.csv")
head(df)

## add sample annotation
condition <- df$condition
names(condition) <- df$sample_id
obj.rna$condition <- stringr::str_replace_all(obj.rna$Sample,
                                                condition)

## add region
region <- df$region
names(region) <- df$sample_id
obj.rna$region <- stringr::str_replace_all(obj.rna$Sample,
                                                region)

## add patient_group
patient_group <- df$patient_group
names(patient_group) <- df$sample_id
obj.rna$patient_group <- stringr::str_replace_all(obj.rna$Sample,
                                                patient_group)

## add global_id
global_id <- df$global_ID
names(global_id) <- df$sample_id
obj.rna$global_id <- stringr::str_replace_all(obj.rna$Sample,
                                                global_id)

## add rep
rep <- as.character(df$rep)
names(rep) <- df$sample_id
obj.rna$rep <- stringr::str_replace_all(obj.rna$Sample,
                                                rep)

## add patient
patient <- df$patient
names(patient) <- df$sample_id
obj.rna$patient <- stringr::str_replace_all(obj.rna$Sample,
                                                patient)

## add patient
region_novel <- df$region_novel
names(region_novel) <- df$sample_id
obj.rna$region_novel <- stringr::str_replace_all(obj.rna$Sample,
                                                region_novel)

## add patient
patient_id <- df$patient_id
names(patient_id) <- df$sample_id
obj.rna$patient_id <- stringr::str_replace_all(obj.rna$Sample,
                                                patient_id)

In [ ]:
## add gene activity matrix
gene.activities <- readRDS("../../DataIntegration/data/VisiumHeart/GeneScoreMatrix.Rds")

dim(gene.activities)

gene.use <- intersect(rownames(gene.activities),
                     rownames(obj.rna))

gene.activities <- gene.activities[gene.use, colnames(obj.atac)]

obj.atac[['GeneActivity']] <- CreateAssayObject(counts = gene.activities)

DefaultAssay(obj.atac) <- "GeneActivity"

obj.atac <- obj.atac %>% 
        NormalizeData() %>%
        FindVariableFeatures() %>%
        ScaleData() %>%
        RunPCA(verbose = FALSE)

In [ ]:
obj.rna <- obj.rna %>% 
        FindVariableFeatures()

In [ ]:
# Identify anchors
transfer.anchors <- FindTransferAnchors(reference = obj.rna, 
                                        query = obj.atac, 
                                        features = VariableFeatures(object = obj.rna),
                                        reference.assay = "RNA", 
                                        query.assay = "GeneActivity", 
                                        reduction = "cca")

In [ ]:
celltype.predictions <- TransferData(anchorset = transfer.anchors, 
                                     refdata = obj.rna$cell_type,
                                     weight.reduction = obj.atac[["harmony"]], 
                                     dims = 1:30)

obj.atac <- AddMetaData(obj.atac, metadata = celltype.predictions)

In [ ]:
p1 <- DimPlot(obj.atac, reduction = "umap_harmony_v2", group.by = "cell_type",
             raster = TRUE, size = 0.1) +
    scale_color_manual(values = cols) +
    ggtitle("snATAC-seq cell type")
    
p2 <- DimPlot(obj.atac, reduction = "umap_harmony_v2", group.by = "predicted.id",
             size = 0.1, raster = TRUE) +
    scale_color_manual(values = cols) +
    ggtitle("snATAC-seq predicted label")

options(repr.plot.height = 5, repr.plot.width = 12)

p1 + p2

In [ ]:
## merge ATAC-RNA
# we here impute full transcriptome
refdata <- GetAssayData(obj.rna, assay = "RNA", slot = "data")[gene.use, ]

# refdata (input) contains a scRNA-seq expression matrix for the scRNA-seq cells.  imputation
# (output) will contain an imputed scRNA-seq matrix for each of the ATAC cells
obj.atac[["RNA"]] <- TransferData(anchorset = transfer.anchors, 
                                  refdata = refdata, 
                                  weight.reduction = "cca")

DefaultAssay(obj.atac) <- "RNA"

obj.rna$tech <- "RNA"
obj.atac$tech <- "ATAC"

obj.rna.sub <- subset(obj.rna, features = gene.use)

In [ ]:
coembed <- merge(x = obj.atac, y = obj.rna.sub)

In [ ]:
#Finally, we run PCA and UMAP on this combined object, to visualize the co-embedding of both datasets
coembed <- coembed %>%
    ScaleData(features = gene.use, do.scale = FALSE) %>%
    FindVariableFeatures() %>%
    RunPCA() %>%
    RunUMAP(dims = 1:30, verbose = FALSE)

In [ ]:
p1 <- DimPlot(coembed, group.by = "tech") +
    xlab("UMAP1") + ylab("UMAP2")

p2 <- DimPlot(coembed, group.by = "cell_type", label = TRUE) +
    xlab("UMAP1") + ylab("UMAP2")

p1 + p2

In [ ]:
saveRDS(obj.atac, file = "../data/snATAC.Rds")
saveRDS(obj.rna, file = "../data/snRNA.Rds")
saveRDS(coembed, file = "../data/snRNA-snATAC-coembedding.Rds")

In [ ]:
sessionInfo()